In [70]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt

In [71]:
file_path = '../data/nba_points_2024_2025.xlsx'
df = pd.read_excel(file_path, usecols="A:T")

# Remove whitespace from Home
df['Home'] = df['Home'].str.strip()
# Remove whitespace from Away
df['Away'] = df['Away'].str.strip()

# Feature engineering
df['Home Win'] = (df['Home Points'] > df['Away Points']).astype(int)
df['Away Win'] = (df['Away Points'] > df['Home Points']).astype(int)

# Assume df is your original DataFrame with a 'Date' column and 'Home' and 'Away' columns
df['Date'] = pd.to_datetime(df['Date'])
# Build a dictionary mapping each team to the set of dates they've played.
teams = pd.concat([df['Home'], df['Away']]).unique()
team_dates = {}
for team in teams:
    # Get all games (dates) where the team appears as home or away.
    team_games = df[(df['Home'] == team) | (df['Away'] == team)]
    # Use dt.date to focus on the date part only.
    team_dates[team] = set(team_games['Date'].dt.date)
# Helper function to check if a team played on the day before the current date.
def played_yesterday(team, current_date):
    # Subtract one day from the current date and compare as a date.
    previous_date = (current_date - pd.Timedelta(days=1)).date()
    return previous_date in team_dates.get(team, set())
# Create new columns using apply:
df['Away BackToBack'] = df.apply(lambda row: played_yesterday(row['Away'], row['Date']), axis=1)
df['Home BackToBack'] = df.apply(lambda row: played_yesterday(row['Home'], row['Date']), axis=1)
df['Either BackToBack'] = df['Away BackToBack'] | df['Home BackToBack']
df['Both BackToBack'] = df['Away BackToBack'] & df['Home BackToBack']

In [72]:
print('Away BackToBacks: ', df['Away BackToBack'].sum(), " | " ,round(df['Away BackToBack'].mean() * 100), '%')
print('Home BackToBacks: ', df['Home BackToBack'].sum(), " | " ,round(df['Home BackToBack'].mean() * 100), '%')
print('Either BackToBacks: ', df['Either BackToBack'].sum(), " | " ,round(df['Either BackToBack'].mean() * 100), '%')
print('Both BackToBacks: ', df['Both BackToBack'].sum(), " | " ,round(df['Both BackToBack'].mean() * 100), '%')

Away BackToBacks:  176  |  19 %
Home BackToBacks:  165  |  18 %
Either BackToBacks:  287  |  31 %
Both BackToBacks:  54  |  6 %


In [73]:
df.columns

Index(['Date', 'Weekday', 'Away', 'Home', 'NBA Cup Game', 'Crew Chief',
       'Referee', 'Umpire', 'Open Total', 'Open Home Spread', 'Away Points',
       'Home Points', 'Actual Total', 'Actual Home Spread', 'Game Count',
       'Over', 'Over Amount', 'Under', 'Under Amount', 'Push', 'Home Win',
       'Away Win', 'Away BackToBack', 'Home BackToBack', 'Either BackToBack',
       'Both BackToBack'],
      dtype='object')

In [74]:
print('Games Tracked: ', round(df['Game Count'].sum()))
print('Average Open Total: ', round(df['Open Total'].mean(), 2))
print('Average Actual Total: ', round(df['Actual Total'].mean(), 2))
print('Over Percent: ', round(df['Over'].mean() * 100, 2))
print('Under Percent: ', round(df['Under'].mean() * 100, 2))
print('First Date Tracked: ', df[df['Game Count'] == 1]['Date'].dt.strftime('%Y-%m-%d').min())
print('Last Date Tracked: ', df[df['Game Count'] == 1]['Date'].dt.strftime('%Y-%m-%d').max())

Games Tracked:  923
Average Open Total:  225.84
Average Actual Total:  226.67
Over Percent:  52.59
Under Percent:  47.08
First Date Tracked:  2024-10-22
Last Date Tracked:  2025-03-05


In [75]:
# Get all unique team names from both Home and Away columns
teams = pd.concat([df['Home'], df['Away']]).unique()

# Initialize a list to store the stats for each team
stats = []

for team in teams:
    # Overall: games where the team is either home or away
    team_games = df[(df['Home'] == team) | (df['Away'] == team)]
    overall_over_pct = team_games['Over'].mean() * 100  # Proportion of games with Over
    
    # Home games only
    home_games = df[df['Home'] == team]
    home_over_pct = home_games['Over'].mean() * 100 if not home_games.empty else None
    
    # Away games only
    away_games = df[df['Away'] == team]
    away_over_pct = away_games['Over'].mean() * 100 if not away_games.empty else None
    
    stats.append({
        'Team': team,
        'Overall Over %': overall_over_pct,
        'Home Over %': home_over_pct,
        'Away Over %': away_over_pct
    })

# Convert the list of stats into a DataFrame
stats_df = pd.DataFrame(stats)

In [76]:
# teams with highest over percentage
print('Teams with the Most Overs:')
print('-' * 50)
print(stats_df.sort_values('Overall Over %', ascending=False).head(round(30*.25))) # top 25%

Teams with the Most Overs:
--------------------------------------------------
   Team  Overall Over %  Home Over %  Away Over %
23  MEM       68.852459    62.500000    75.862069
5   ATL       61.290323    68.965517    54.545455
17  CLE       60.655738    60.606061    60.714286
9   UTA       60.317460    54.545455    66.666667
14  DEN       59.677419    66.666667    53.125000
18  NYK       58.333333    56.250000    60.714286
3   PHI       57.377049    53.125000    62.068966
7   NOP       57.377049    54.838710    60.000000


In [77]:
# teams with highest over percentage
print('Teams with the Most Unders:')
print('-' * 50)
print(stats_df.sort_values('Overall Over %', ascending=True).head(round(30*.25))) # top 25%

Teams with the Most Unders:
--------------------------------------------------
   Team  Overall Over %  Home Over %  Away Over %
20  CHA       37.704918    34.375000    41.379310
16  ORL       41.538462    33.333333    50.000000
27  BKN       43.333333    39.285714    46.875000
29  GSW       43.548387    36.666667    50.000000
0   BOS       45.161290    51.612903    38.709677
11  LAC       46.774194    46.666667    46.875000
24  SAS       48.333333    50.000000    46.666667
13  DAL       49.206349    37.500000    61.290323


In [78]:
# Ensure the 'Date' column is datetime
df['Date'] = pd.to_datetime(df['Date'])

# Get all unique team names from both Home and Away columns
teams = pd.concat([df['Home'], df['Away']]).unique()

# List to hold stats for each team
stats = []

for team in teams:
    # Overall games for the team (as home or away)
    team_games = df[(df['Home'] == team) | (df['Away'] == team)]
    team_games_last10 = team_games.sort_values('Date').tail(10)
    
    # Home games for the team
    home_games = df[df['Home'] == team]
    home_games_last10 = home_games.sort_values('Date').tail(10)
    
    # Away games for the team
    away_games = df[df['Away'] == team]
    away_games_last10 = away_games.sort_values('Date').tail(10)
    
    # Calculate Over % for each subset
    overall_over_pct = team_games_last10['Over'].mean() * 100 if not team_games_last10.empty else None
    home_over_pct = home_games_last10['Over'].mean() * 100 if not home_games_last10.empty else None
    away_over_pct = away_games_last10['Over'].mean() * 100 if not away_games_last10.empty else None

    # Calculate scoring averages
    # For overall games, pick the appropriate points column based on whether the team was home or away.
    def get_team_points(row):
        return row['Home Points'] if row['Home'] == team else row['Away Points']
    
    if not team_games_last10.empty:
        overall_team_points = team_games_last10.apply(get_team_points, axis=1)
        overall_avg_points = overall_team_points.mean()
    else:
        overall_avg_points = None

    home_avg_points = home_games_last10['Home Points'].mean() if not home_games_last10.empty else None
    away_avg_points = away_games_last10['Away Points'].mean() if not away_games_last10.empty else None
    overall_avg_total = team_games_last10['Actual Total'].mean()
    overall_avg_open_total = team_games_last10['Open Total'].mean()
    
    stats.append({
        'Team': team,
        'Overall Over % Last10': overall_over_pct,
        'Home Over % Last10': home_over_pct,
        'Away Over % Last10': away_over_pct
        #'Overall Avg Points (last10)': overall_avg_points,
        #'Home Avg Points (last10)': home_avg_points,
        #'Away Avg Points (last10)': away_avg_points,
        #'Overall Avg Total Last10': overall_avg_total,
        #'Overall Avg Open Last10': overall_avg_open_total,
    })

# Convert the list of stats into a DataFrame and display
stats10_df = pd.DataFrame(stats)

In [79]:
print('Teams with the most Overs in their last 10 games:')
print('-'*70)
print(stats10_df.sort_values(by='Overall Over % Last10',ascending=False).head(8))

Teams with the most Overs in their last 10 games:
----------------------------------------------------------------------
   Team  Overall Over % Last10  Home Over % Last10  Away Over % Last10
9   UTA                   80.0                80.0                70.0
10  POR                   70.0                50.0                60.0
28  OKC                   70.0                80.0                60.0
17  CLE                   70.0                70.0                60.0
11  LAC                   70.0                70.0                70.0
5   ATL                   70.0                60.0                70.0
22  CHI                   60.0                70.0                40.0
21  MIN                   60.0                50.0                80.0


In [80]:
# Group by 'Crew Chief' and calculate required stats
crew_stats = df.groupby('Crew Chief').agg(
    games_count=('Crew Chief', 'size'), # total number of games for each crew chief
    over_count=('Over', lambda x: x.sum()),
    under_count=('Under', lambda x: x.sum())
).reset_index()

# Calculate over percentage
crew_stats['over_percentage'] = (crew_stats['over_count'] / crew_stats['games_count']) * 100
crew_stats['under_percentage'] = (crew_stats['under_count'] / crew_stats['games_count']) * 100

In [81]:
print('Crew Chiefs with the Most UNDERS (min 15 games)')
print('--' * 28)
print(crew_stats[crew_stats['games_count']>=15][['Crew Chief',
                                                 'games_count',
                                                 'under_percentage']].sort_values(by='under_percentage',
                                                                                  ascending=False).head(7))

Crew Chiefs with the Most UNDERS (min 15 games)
--------------------------------------------------------
             Crew Chief  games_count  under_percentage
9           Jacyn Goble           16         68.750000
7             Ed Malloy           45         62.222222
11       James Williams           47         59.574468
6         David Guthrie           19         57.894737
22           Pat Fraher           35         54.285714
8   Gediminas Petraitis           24         54.166667
1          Bill Kennedy           26         53.846154


In [82]:
print('Crew Chiefs with the Most OVERS (min 15 games)')
print('--' * 28)
print(crew_stats[crew_stats['games_count']>=15][['Crew Chief',
                                                 'games_count',
                                                 'over_percentage']].sort_values(by='over_percentage',
                                                                                  ascending=False).head(7))

Crew Chiefs with the Most OVERS (min 15 games)
--------------------------------------------------------
        Crew Chief  games_count  over_percentage
28      Tyler Ford           50        66.000000
24    Scott Foster           42        64.285714
13      Josh Tiven           49        63.265306
18      Marc Davis           46        60.869565
20  Mitchell Ervin           35        57.142857
19    Mark Lindsay           37        56.756757
3      Brian Forte           30        56.666667
